# Segmenting and Clustering Neighborhoods in Toronto

### Assignment for the week 3 of the Coursera course"Applied Data Science Capstone"

Import requests package to get the wikipedia page and the BeautifulSoup module for scraping the web page

In [1]:
import requests
from bs4 import BeautifulSoup

Get the wikipedia page

In [2]:
link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(link)

Transform the page in BeautifulSoup object for scraping

In [3]:
soup=BeautifulSoup(page.content,'html.parser')
#print(soup.prettify)

Get the table with the data

In [4]:
table=list(soup.find_all('table',class_='wikitable sortable'))[0]
#table

Get the list of the rows of the table

In [5]:
tr_list=table.find_all('tr')
r =tr_list.pop(0) # delete the firs element, containing the column labels
#tr_list

Bulild a list of dictionaries with the data following the requirenments of the asseignment

In [6]:
data=[]
pcode_prev = ''
for i, row in enumerate(tr_list):                  # iterate on the list of rows
    td_list = row.find_all('td')                   # build a list of the cells in the row
    borough = td_list[1].get_text()                # extract the value of borough in string format
    neighborhood = td_list[2].get_text().rstrip()  # extract the value of neighborhood in string format removing the "\n" characters
    if neighborhood == "Not assigned":
        neighborhood = borough
    if i == 0:                                     # to initialize the variables neighborhoods and borough_prev
        neighborhoods = neighborhood
        borough_prev = borough
    if( borough != "Not assigned"):
        pcode = td_list[0].get_text()        # current value of postal code
        if i == 0:
            data.append({'PostalCode': pcode,'Borough': borough,'Neighborhood': neighborhoods})
        else:
            if( pcode == pcode_prev):        # I'm assuming rows with the same postal code are in successive order 
                neighborhoods = neighborhoods + ', ' + neighborhood   # concatenate the neighborhood names
            else:                            # when find different postal code then append the row with the values of the previous postal code
                data.append({'PostalCode': pcode_prev,'Borough': borough_prev,'Neighborhood': neighborhoods})
                neighborhoods = neighborhood
        pcode_prev = pcode                  # register the previous vaule of postal code
        borough_prev = borough

Import pandas package

In [7]:
import pandas as pd

#### Build the dataframe from the list of dictionaries just built.

In [8]:
df=pd.DataFrame(data)
df=df[["PostalCode",'Borough','Neighborhood']]
df=df.drop(df.index[0])         # remove the first line
df = df.reset_index(drop=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


#### Dataframe shape:

In [9]:
df.shape

(102, 3)

There are 102 postal codes with assigned borough.

## Second part of the assignment

I have tried to use the Geocoder package but it doesn't work. So I download the csv file.

In [10]:
!wget -O toronto_geospatialdata.csv https://cocl.us/Geospatial_data

--2018-12-07 12:35:10--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2018-12-07 12:35:12--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.26.197
Connecting to ibm.box.com (ibm.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2018-12-07 12:35:13--  https://ibm.ent.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.ent.box.com (ibm.ent.box.com)... 107.152.26.211
Connecting to ibm.ent.box.com (ibm.ent.box.com)|107.152.26.211|:443... connected.
HTTP request sent, awaiting response... 302 Found

#### Load the data in a dataframe

In [11]:
toronto_geo_df = pd.read_csv('toronto_geospatialdata.csv')
toronto_geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Make a join between the two dataframe on 'PostalCode' column

In [12]:
toronto_df = df
toronto_df = toronto_df.join(toronto_geo_df.set_index('Postal Code'),on='PostalCode')
toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
